<a href="https://colab.research.google.com/github/filmerxyz/JSTP-22_SkinDiseaseClassificationUsingMachineLearning/blob/master/Copy_of_Almost_Accumulator_DenseNet201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU

In [1]:
!nvidia-smi

Sun May 24 15:47:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os
import math
import itertools
from PIL import Image
from matplotlib import pyplot as plt

import keras
from keras import backend as K
from keras.optimizers import Adam, Optimizer
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.legacy import interfaces

from sklearn.model_selection import train_test_split
from sklearn.utils import compute_sample_weight, compute_class_weight
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix

Using TensorFlow backend.


In [3]:
!git clone https://github.com/EvilPickle-PCSHSPT/ham10000-with-one-image-folder

Cloning into 'ham10000-with-one-image-folder'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10045 (delta 4), reused 7 (delta 2), pack-reused 10036
Receiving objects: 100% (10045/10045), 2.57 GiB | 43.79 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Checking out files: 100% (10022/10022), done.


In [0]:
metadata_path = '/content/ham10000-with-one-image-folder/HAM10000_metadata.csv'
img_path = '/content/ham10000-with-one-image-folder/HAM1000_images'

In [0]:
label_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [0]:
data = pd.read_csv(metadata_path)

In [0]:
num_examples = data.values.shape[0]

In [0]:
inputs = np.empty(shape=(num_examples, 224, 224, 3), dtype=np.float32)
labels = np.empty(shape=(num_examples), dtype=np.uint8)

### Resize to 224*224

In [0]:
for i, row in enumerate(data.values):
    img_id = row[1]
    label = row[2]
    all_img_path = os.path.join(img_path, img_id) + '.jpg' 
    if (os.path.isfile(all_img_path)):
        im_path = all_img_path
    else:
        raise Exception ('File not found \'%s\'' %img_id)
    img = Image.open(im_path).resize((224, 224), Image.LANCZOS)
    inputs[i] = np.array(img)/255.0
    labels[i] = label_names.index(label)

### Split Data

In [0]:
x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.1, random_state=2019)

del inputs
del labels

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=2019)

In [11]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(6309, 224, 224, 3) (6309,)
(2704, 224, 224, 3) (2704,)
(1002, 224, 224, 3) (1002,)


# Hyperparameters

In [0]:
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 100

# Data Augmentation

In [0]:
datagen = ImageDataGenerator(
        rotation_range=360,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='reflect')

In [0]:
datagen.fit(x_train)

# Adam Accumulate

In [15]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import tensorflow as tf

# Build Model

In [17]:
base_model = keras.applications.DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(7,activation='softmax')(x)

model = Model(inputs=base_model.input,outputs=preds)

model.compile(optimizer=Adam(LR), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

74842112/74836368 [==============================] - 1s 0us/step


In [0]:
# model.summary()

### Callbacks

In [0]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='acc_densenet201_model.{epoch:02d}-{val_loss:.2f}.h5', save_best_only=True, monitor='val_loss', mode='min')

reduce_lr_rate = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, min_lr=0.00001 , patience=10, verbose=1, min_delta=1e-4, mode='min')

### Training

In [20]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(x_val, y_val, compute_sample_weight('balanced', y_val)),
                              class_weight=compute_class_weight('balanced', np.unique(y_train), y_train),
                              steps_per_epoch=math.ceil(x_train.shape[0]/BATCH_SIZE),
                              epochs=EPOCHS,
                              callbacks=[early_stop, model_checkpoint, reduce_lr_rate])

Epoch 1/100
395/395 [==============================] - 318s 806ms/step - loss: 0.9383 - accuracy: 0.6822 - val_loss: 8.7290 - val_accuracy: 0.1934
Epoch 2/100
395/395 [==============================] - 218s 551ms/step - loss: 0.8710 - accuracy: 0.6874 - val_loss: 5.5136 - val_accuracy: 0.4982
Epoch 3/100
395/395 [==============================] - 216s 547ms/step - loss: 0.8331 - accuracy: 0.6987 - val_loss: 2.0462 - val_accuracy: 0.7004
Epoch 4/100
395/395 [==============================] - 218s 552ms/step - loss: 0.7695 - accuracy: 0.7207 - val_loss: 1.9287 - val_accuracy: 0.7145
Epoch 5/100
395/395 [==============================] - 217s 550ms/step - loss: 0.7413 - accuracy: 0.7280 - val_loss: 2.0028 - val_accuracy: 0.7152
Epoch 6/100
395/395 [==============================] - 218s 551ms/step - loss: 0.7217 - accuracy: 0.7334 - val_loss: 38.9930 - val_accuracy: 0.6842
Epoch 7/100
395/395 [==============================] - 217s 548ms/step - loss: 0.7177 - accuracy: 0.7336 - val_loss: 